In [72]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 900)
pd.set_option('display.max_columns', 900)
pd.set_option('display.width', 850)

In [73]:
def add_new_columns(state_df):
    state_df['sum_r12'] = ''
    state_df['sum_r345'] = ''
    state_df['relocate'] = ''
    state_df['relocating_provnum'] = ''
    state_df['relocating_provname'] = ''
    return state_df

In [74]:
df = pd.read_csv("data/homes_example.csv")

In [75]:
df = df.drop(columns=['Count of FACID']) 
df = df.fillna(method='ffill')
df['Comp5star'] = df['Comp5star'].astype(int)
df['PROVNUM']   = df['PROVNUM'].astype(str)
df['PROVNAME']  = df['PROVNAME'].astype(str)

#df

In [81]:
def process_county(county_df):
    return county_df

In [80]:
l_df_out = []

states = df.STATE.unique()

for state in states:
    #print("/////////////////////////////////////////", state, "/////////////////////////////////")
    state_df = df.loc[df['STATE'] == state]
    state_df = add_new_columns(state_df)
    #print(state_df)
    
    # get all counties for this state
    counties_in_state = state_df.County_name.unique()
    #print(counties_in_state)

    for county in counties_in_state:
        #print("-----------------------------------------", county, "------------------------------")
        county_df = state_df.loc[state_df['County_name'] == county]
        
        county_processed_df = process_county(county_df)
        #print(county_processed_df)
        
        l_df_out.append(county_processed_df)
        #print("-----------------------------------------", county, "------------------------------")
    #print("/////////////////////////////////////////", state, "/////////////////////////////////\n")

df_out = (pd.concat(l_df_out)).reset_index(drop=True)
print(df_out.to_string())
df_out.to_csv("df_out.csv", index=False)

   STATE           County_name  Comp5star PROVNUM PROVNAME  Sum of MDScensus_avg  Sum of Beds_Avail sum_r12 sum_r345 relocate relocating_provnum relocating_provname
0     AK             Anchorage          4   25018        A                    10                 16                                                                 
1     AK             Anchorage          5   25025        B                    11                 17                                                                 
2     AK             Anchorage          5   25036        C                    12                 18                                                                 
3     AK  Fairbanks North Star          3   25020        D                    13                 19                                                                 
4     WY               Laramie          1  535013        E                    17                 28                                                                 
5     WY  